In [ ]:
# !pip install sentence_transformers
# !pip install keras_tuner
# !pip install python-louvain

# from google.colab import drive
# drive.mount('/content/drive')


import csv
import json
import re
import ast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics.cluster import normalized_mutual_info_score
from sentence_transformers import SentenceTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import NearestCentroid
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix,accuracy_score,ConfusionMatrixDisplay,precision_score,recall_score,f1_score,pairwise_distances
from sklearn.preprocessing import normalize
from imblearn.under_sampling import AllKNN
from imblearn.under_sampling import RandomUnderSampler
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
import plotly.graph_objects as go
import plotly.express as px
import networkx as nx
import networkx.algorithms.community as nx_comm
from community import community_louvain
from ast import literal_eval
np.set_printoptions(linewidth=25000)

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/TCC/df_train_new.txt')
df_test = pd.read_csv('/content/drive/MyDrive/TCC/df_test_new.txt')

def limpa(x):
    return [float(n) for n in (x.strip('[]').replace(' ',',').replace(',,',',').split(',')) if n != '' ]

df_train['Embedding'] = df_train['Embedding'].apply(lambda x: limpa(x))
df_test['Embedding'] = df_test['Embedding'].apply(lambda x: limpa(x))

def limpa2(x):
    return int(x.strip('[]'))

df_train['Genre_Class'] = df_train['Genre_Class'].apply(lambda x: limpa2(x))
df_test['Genre_Class'] = df_test['Genre_Class'].apply(lambda x: limpa2(x))

print(df_train['Genre_Class'].value_counts())
print(df_test['Genre_Class'].value_counts())

def lista(x):
    return [x]

ohe = OneHotEncoder(sparse=False)
ohe_train = pd.Series(list(ohe.fit_transform(df_train['Genre_Class'].apply(lambda x: lista(x)).tolist())))
ohe_test = pd.Series(list(ohe.fit_transform(df_test['Genre_Class'].apply(lambda x: lista(x)).tolist())))

df_train['ohe_train'] = ohe_train
df_test['ohe_test'] = ohe_test

df_train.info()
df_test.info()

plt.hist(df_train['Genre_Class'],[-0.5,0.5,1.5,2.5,3.5,4.5],rwidth=0.85)
plt.xlabel('Gênero')
plt.ylabel('Quantidade')
plt.title("Distribuição da quantidade de dados de cada gênero\nConjunto de Treino")
plt.show()

plt.hist(df_test['Genre_Class'],[-0.5,0.5,1.5,2.5,3.5,4.5],rwidth=0.85)
plt.xlabel('Gênero')
plt.ylabel('Quantidade')
plt.title('Distribuição da quantidade de dados de cada gênero\nConjunto de Teste')
plt.show()

In [ ]:
rus = RandomUnderSampler(random_state=666)
df_train_rus,_ = rus.fit_resample(df_train,df_train['Genre_Class'].tolist())
print(df_train_rus['Genre_Class'].value_counts())

df_train = df_train_rus.copy()

plt.hist(df_train_rus['Genre_Class'],[-0.5,0.5,1.5,2.5,3.5,4.5],rwidth=0.85)
plt.xlabel('Gênero')
plt.ylabel('Quantidade')
plt.title('Distribuição da quantidade de dados de cada gênero\nConjunto de Treino após Subsampling')
plt.show()

In [ ]:
n = 13
train_classifier = KNeighborsClassifier(n_neighbors = n, metric = 'euclidean')
train_classifier.fit(np.array(df_train['Embedding'].tolist()), np.array(df_train['Genre_Class'].tolist()))


In [ ]:
train_graph = nx.Graph(train_classifier.kneighbors_graph())

pos = nx.spring_layout(train_graph, seed=42)

for node in train_graph.nodes():
    train_graph.nodes[node]['pos'] = pos[node]


def show_graph(G,color):
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)
    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=2, color='#888'),
        hoverinfo='none',
        mode='lines')
    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = G.nodes[node]['pos']
        node_x.append(x)
        node_y.append(y)
    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        hovertext=df_train['BookTitle'].tolist(),
        marker=dict(
            size=10,
            line_width=2,
            color=color))
    fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40),
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                    )
    fig.show()
    

In [ ]:
show_graph(train_graph, df_train['Genre_Class'].tolist())

In [ ]:
#for n in np.arange(0.6,0.85,0.01):
n = 0.70
train_communities = community_louvain.best_partition(train_graph,random_state=42,resolution=n)

train_nmi = normalized_mutual_info_score(df_train['Genre_Class'].tolist(), list(train_communities.values()))
print(n," - ",train_nmi," - ",len(set(list(train_communities.values()))))


df_train['Genre_Class_NMI'] = pd.Series(list(train_communities.values()))

mapa = []
for nmi in [0,1,2,3,4]:
    mapa.append(df_train['Genre_Class_NMI']==nmi)

show_graph(train_graph, df_train['Genre_Class_NMI'].tolist())

In [ ]:
n = 13
train_classifier_NMI = KNeighborsClassifier(n_neighbors = n, metric = 'euclidean')
train_classifier_NMI.fit(np.array(df_train['Embedding'].tolist()), np.array(df_train['Genre_Class_NMI'].tolist()))

y_pred_NMI = train_classifier_NMI.predict(np.array(df_test['Embedding'].tolist()))


cm_NMI = confusion_matrix(np.array(df_test['Genre_Class'].tolist()), y_pred_NMI)
ac_NMI = accuracy_score(np.array(df_test['Genre_Class'].tolist()),y_pred_NMI)



print(cm_NMI)
print(ac_NMI)

In [ ]:
rus_test = RandomUnderSampler(random_state=666)
df_test_rus,_ = rus_test.fit_resample(df_test,df_test['Genre_Class'].tolist())

print(df_test_rus['Genre_Class'].value_counts())

y_pred_NMI = train_classifier_NMI.predict(np.array(df_test_rus['Embedding'].tolist()))


cm_NMI = confusion_matrix(np.array(df_test_rus['Genre_Class'].tolist()), y_pred_NMI)
ac_NMI = accuracy_score(np.array(df_test_rus['Genre_Class'].tolist()),y_pred_NMI)

print(cm_NMI)
print(ac_NMI)

print(precision_score(np.array(df_test_rus['Genre_Class'].tolist()), y_pred, average=None))
print(recall_score(np.array(df_test_rus['Genre_Class'].tolist()), y_pred, average=None))
print(f1_score(np.array(df_test_rus['Genre_Class'].tolist()), y_pred, average=None))

disp = ConfusionMatrixDisplay.from_predictions(np.array(df_test_rus['Genre_Class'].tolist()), y_pred_NMI, display_labels=selected_genres, xticks_rotation=60, cmap="gist_yarg")


In [ ]:
train_centroid = NearestCentroid()
train_centroid.fit(np.array(df_train['Embedding'].tolist()), np.array(df_train['Genre_Class'].tolist()))
train_centroid.score(np.array(df_test['Embedding'].tolist()), np.array(df_test['Genre_Class'].tolist()))


sumario_foundation = """For twelve thousand years the Galactic Empire has ruled supreme. Now it is dying. But only Hari Seldon, creator of the revolutionary science of psychohistory, can see into the future -- to a dark age of ignorance, barbarism, and warfare that will last thirty thousand years. To preserve knowledge and save mankind, Seldon gathers the best minds in the Empire both scientists and scholars and brings them to a bleak planet at the edge of the Galaxy to serve as a beacon of hope for a future generations. He calls his sanctuary the Foundation.But soon the fledgling Foundation finds itself at the mercy of corrupt warlords rising in the wake of the receding Empire. Mankind's last best hope is faced with an agonizing choice: submit to the barbarians and be overrun or fight them and be destroyed."""
print(len(sumario_foundation.split()))

sumario_narnia = """Narnia… the land beyond the wardrobe door, a secret place frozen in eternal winter, a magical country waiting to be set free. Lucy is the first to find the secret of the wardrobe in the professor's mysterious old house. At first her brothers and sister don't believe her when she tells of her visit to the land of Narnia. But soon Edmund, then Peter and Susan step through the wardrobe themselves. In Narnia they find a country buried under the evil enchantment of the White Witch. When they meet the Lion Aslan, they realize they've been called to a great adventure and bravely join the battle to free Narnia from the Witch's sinister spell."""
print(len(sumario_narnia.split()))

sumario_tbp = """Set against the backdrop of China's Cultural Revolution, a secret military project sends signals into space to establish contact with aliens. An alien civilization on the brink of destruction captures the signal and plans to invade Earth. Meanwhile, on Earth, different camps start forming, planning to either welcome the superior beings and help them take over a world seen as corrupt, or to fight against the invasion."""

sumario_HP = """Harry Potter's life is miserable. His parents are dead and he's stuck with his heartless relatives, who force him to live in a tiny closet under the stairs. But his fortune changes when he receives a letter that tells him the truth about himself: he's a wizard. A mysterious visitor rescues him from his relatives and takes him to his new home, Hogwarts School of Witchcraft and Wizardry. After a lifetime of bottling up his magical powers, Harry finally feels like a normal kid. But even within the Wizarding community, he is special. He is the boy who lived: the only person to have ever survived a killing curse inflicted by the evil Lord Voldemort, who launched a brutal takeover of the Wizarding world, only to vanish after failing to kill Harry. Though Harry's first year at Hogwarts is the best of his life, not everything is perfect. There is a dangerous secret object hidden within the castle walls, and Harry believes it's his responsibility to prevent it from falling into evil hands. But doing so will bring him into contact with forces more terrifying than he ever could have imagined."""

sumario_lotr = """One Ring to rule them all, One Ring to find them, One Ring to bring them all and in the darkeness bind them. In ancient times the Rings of Power were crafted by the Elven-smiths, and Sauron, The Dark Lord, forged the One Ring, filling it with his own power so that he could rule all others. But the One Ring was taken from him, and though he sought it throughout Middle-earth, it remained lost to him. After many ages it fell into the hands of Bilbo Baggins, as told in The Hobbit. In a sleepy village in the Shire, young Frodo Baggins finds himself faced with an immense task, as his elderly cousin Bilbo entrusts the Ring to his care. Frodo must leave his home and make a perilous journey across Middle-earth to the Cracks of Doom, there to destroy the Ring and foil the Dark Lord in his evil purpose."""

sumario_dune = """Set on the desert planet Arrakis, Dune is the story of the boy Paul Atreides, heir to a noble family tasked with ruling an inhospitable world where the only thing of value is the “spice” melange, a drug capable of extending life and enhancing consciousness. Coveted across the known universe, melange is a prize worth killing for. When House Atreides is betrayed, the destruction of Paul’s family will set the boy on a journey toward a destiny greater than he could ever have imagined. And as he evolves into the mysterious man known as Muad’Dib, he will bring to fruition humankind’s most ancient and unattainable dream."""

sumario_blade = """It was January 2021, and Rick Deckard had a license to kill. Somewhere among the hordes of humans out there, lurked several rogue androids. Deckard's assignment--find them and then..."retire" them. Trouble was, the androids all looked exactly like humans, and they didn't want to be found!"""


model = SentenceTransformer('/content/drive/MyDrive/TCC/finetune-test')
model.max_seq_length = 512

emb_foundation = model.encode(sumario_foundation)
emb_narnia = model.encode(sumario_narnia)
emb_tbp = model.encode(sumario_tbp)
emb_HP = model.encode(sumario_HP)
emb_lotr = model.encode(sumario_lotr)
emb_dune = model.encode(sumario_dune)
emb_blade = model.encode(sumario_blade)

# print(train_centroid.predict([emb_foundation,emb_narnia]))
# print(train_centroid.centroids_)

#print(pairwise_distances(train_centroid.centroids_,[emb_foundation,emb_narnia]))
dist_foundation = normalize(pairwise_distances(train_centroid.centroids_,emb_foundation.reshape(1,-1)), axis=0)
dist_narnia = normalize(pairwise_distances(train_centroid.centroids_,emb_narnia.reshape(1,-1)), axis=0)
dist_tbp = normalize(pairwise_distances(train_centroid.centroids_,emb_tbp.reshape(1,-1)), axis=0)
dist_HP = normalize(pairwise_distances(train_centroid.centroids_,emb_HP.reshape(1,-1)), axis=0)
dist_lotr = normalize(pairwise_distances(train_centroid.centroids_,emb_lotr.reshape(1,-1)), axis=0)
dist_dune = normalize(pairwise_distances(train_centroid.centroids_,emb_dune.reshape(1,-1)), axis=0)
dist_blade = normalize(pairwise_distances(train_centroid.centroids_,emb_blade.reshape(1,-1)), axis=0)

# print(dist_foundation)
# print(dist_narnia)

print(np.argmin(dist_foundation))
print(np.argmin(dist_narnia))
print(np.argmin(dist_tbp))
print(np.argmin(dist_HP))
print(np.argmin(dist_lotr))
print(np.argmin(dist_dune))
print(np.argmin(dist_blade))
#print(1-np.ravel(dist_foundation))

print((1-np.ravel(dist_foundation)))
print((1-np.ravel(dist_narnia)))
print((1-np.ravel(dist_blade)))

fig = px.line_polar(r=(1-np.ravel(dist_foundation)), theta=selected_genres, line_close=True, range_r = [0,1], title="Componentes gêneros: Fundação (I. Asimov)")
fig.update_traces(fill='toself')
fig.update_layout(title={'y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'})
fig.show()

fig = px.line_polar(r=(1-np.ravel(dist_narnia)), theta=selected_genres, line_close=True, range_r = [0,1], title="Componentes gêneros: Narnia (C. S. Lewis)")
fig.update_traces(fill='toself')
fig.update_layout(title={'y':0.9,'x':0.5,'xanchor':'center','yanchor': 'top'})
fig.show()

# fig = px.line_polar(r=(1-np.ravel(dist_tbp)), theta=selected_genres, line_close=True, range_r = [0,1], title="Componentes gêneros: The Three-Body Problem (Liu Cixin)")
# fig.update_traces(fill='toself')
# fig.update_layout(title={'y':0.9,'x':0.5,'xanchor':'center','yanchor': 'top'})
# fig.show()

# fig = px.line_polar(r=(1-np.ravel(dist_HP)), theta=selected_genres, line_close=True, range_r = [0,1], title="Componentes gêneros: Harry Potter and the Sorcerer's Stone (J.K. Rowling)")
# fig.update_traces(fill='toself')
# fig.update_layout(title={'y':0.9,'x':0.5,'xanchor':'center','yanchor': 'top'})
# fig.show()

# fig = px.line_polar(r=(1-np.ravel(dist_lotr)), theta=selected_genres, line_close=True, range_r = [0,1], title="Componentes gêneros: The Fellowship of the Ring (J.R.R. Tolkien)")
# fig.update_traces(fill='toself')
# fig.update_layout(title={'y':0.9,'x':0.5,'xanchor':'center','yanchor': 'top'})
# fig.show()

# fig = px.line_polar(r=(1-np.ravel(dist_dune)), theta=selected_genres, line_close=True, range_r = [0,1], title="Componentes gêneros: Dune (Frank Herbert)")
# fig.update_traces(fill='toself')
# fig.update_layout(title={'y':0.9,'x':0.5,'xanchor':'center','yanchor': 'top'})
# fig.show()

fig = px.line_polar(r=(1-np.ravel(dist_blade)), theta=selected_genres, line_close=True, range_r = [0,1], title="Componentes gêneros: Do Androids Dream of Electric Sheep? (Philip K. Dick)")
fig.update_traces(fill='toself')
fig.update_layout(title={'y':0.9,'x':0.5,'xanchor':'center','yanchor': 'top'})
fig.show()